In [2]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv


os.makedirs("reports", exist_ok=True)


# Load credentials
load_dotenv()
engine = create_engine(
    f"postgresql+psycopg2://{os.getenv('PG_USER')}:{os.getenv('PG_PASSWORD')}@{os.getenv('PG_HOST')}:{os.getenv('PG_PORT')}/{os.getenv('PG_DB')}"
)
pd.set_option('display.max_rows', None)


In [ ]:
# Descriptive Business Question: What are the top 10 keywords with the highest search volume but low competition?

In [4]:
sql_query = '''
WITH recent_keywords AS (
    SELECT *
    FROM sql_project.api_keywords
    WHERE scraped_at = (SELECT MAX(scraped_at) FROM sql_project.api_keywords)
)
SELECT 
    keyword,
    vol,
    cpc::float8 AS cpc,
    competition,
    score
FROM recent_keywords
WHERE LOWER(competition) = 'low'
ORDER BY vol DESC
LIMIT 10;
'''


In [7]:
df = pd.read_sql(sql_query, engine)
pd.set_option('display.max_rows', None)
df 

,keyword,day,vol,score,cpc,time_rank
0,email campaigns,2025-05-12,1900,0.5625,2.25,1
1,email image,2025-05-12,3600,0.0850,0.34,1
2,emailing marketing,2025-05-12,27100,0.3600,1.44,1
3,email marketing,2025-05-12,27100,0.3600,1.44,1
4,email marketing templates,2025-05-12,1900,0.2175,0.87,1
5,email marketing tools,2025-05-12,6600,0.7150,2.86,1
6,image for email,2025-05-12,3600,0.0850,0.34,1
7,marketing emails,2025-05-12,22200,0.3600,1.44,1
8,what is email advertising,2025-05-12,6600,0.1050,0.42,1
9,what is email marketing,2025-05-12,6600,0.1050,0.42,1


In [ ]:
# Diagnostic Business Question: Why do some keywords with extremely high search volume still show uniformly low CPC — and are there early signs of rising competition or value?

In [6]:
# Diagnostic query
sql_query = '''
WITH latest_keywords AS (
    SELECT *
    FROM sql_project.api_keywords
    WHERE scraped_at = (SELECT MAX(scraped_at) FROM sql_project.api_keywords)
      AND LOWER(competition) = 'low'
    ORDER BY vol DESC
    LIMIT 10
),
top_keywords AS (
    SELECT keyword
    FROM latest_keywords
),
keyword_trends AS (
    SELECT
        k.keyword,
        k.scraped_at::date AS day,
        k.vol,
        k.score,
        k.cpc::float8 AS cpc,
        RANK() OVER (PARTITION BY k.keyword ORDER BY k.scraped_at) AS time_rank
    FROM sql_project.api_keywords k
    JOIN top_keywords t
        ON k.keyword = t.keyword
)
SELECT *
FROM keyword_trends
ORDER BY keyword, day;
'''

# Save to DataFrame
df_diagnostic = pd.read_sql(sql_query, engine)
pd.set_option('display.max_rows', None)

# Display the DataFrame
df_diagnostic

,keyword,day,vol,score,cpc,time_rank
0,email campaigns,2025-05-12,1900,0.5625,2.25,1
1,email image,2025-05-12,3600,0.0850,0.34,1
2,emailing marketing,2025-05-12,27100,0.3600,1.44,1
3,email marketing,2025-05-12,27100,0.3600,1.44,1
4,email marketing templates,2025-05-12,1900,0.2175,0.87,1
5,email marketing tools,2025-05-12,6600,0.7150,2.86,1
6,image for email,2025-05-12,3600,0.0850,0.34,1
7,marketing emails,2025-05-12,22200,0.3600,1.44,1
8,what is email advertising,2025-05-12,6600,0.1050,0.42,1
9,what is email marketing,2025-05-12,6600,0.1050,0.42,1


In [ ]:
#Diagnostic Analysis --

# Insight: Despite wide variation in search volume, all top keywords share the same CPC — likely due to lag in advertiser targeting or undetected emerging trends.

#Recommendation: Investigate trending score, SERP saturation, or conversion rates for these keywords over time. Use that data to preemptively claim ad space before competition and CPC rise.

#Prediction: High-volume keywords will likely attract more bidding pressure soon. Expect CPC differentiation to emerge as advertisers begin to prioritize based on volume-performance ratios.

In [ ]:
#Descriptive Analysis --

#Insight - These keywords are highly searched but face little competition, making them great for content or ad targeting.

#Recommendation - Prioritize these for SEO blog posts or cost-efficient ad bids before competitors do.

#Prediction - Some of these low-competition terms may become more competitive as others discover them — track their CPC over time.